In [3]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
from smount_predictors import SeamountHelp
from smount_predictors.src.SeamountHelp import PipelinePredictor
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
import simplekml
import pygmt

In [5]:
ages = xr.open_dataset('data/age.2020.1.GTS2012.1m.nc')
mounts = xr.open_dataset('out/global_predictions.nc')
mounts = mounts.to_dataframe().set_index(['lon', 'lat'])
xr.Dataset.from_dataframe(mounts).to_netcdf('data/global_predictions_indexed.nc')

In [6]:
mounts = xr.open_dataset('out/global_predictions_indexed.nc')

In [7]:
# Regrid the 'ages' dataset to match the grid of 'mounts'
ages_regridded = ages.interp(lon=mounts.lon, lat=mounts.lat)
ages_regridded = ages_regridded.rename({'z': 'age'})
# Merge the datasets
merged_dataset = xr.merge([mounts, ages_regridded])
merged_dataset

<xarray.Dataset> Size: 22GB
Dimensions:  (lon: 43200, lat: 18600)
Coordinates:
  * lon      (lon) float64 346kB -1.031e+04 -1.031e+04 ... 1.031e+04 1.031e+04
  * lat      (lat) float64 149kB -4.583e+03 -4.582e+03 ... 4.582e+03 4.583e+03
Data variables:
    z        (lon, lat) float32 3GB ...
    class    (lon, lat) float64 6GB ...
    cluster  (lon, lat) float64 6GB ...
    age      (lat, lon) float64 6GB nan nan nan nan nan ... nan nan nan nan nan

In [8]:
merged_dataset['age']

<xarray.DataArray 'age' (lat: 18600, lon: 43200)> Size: 6GB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lon      (lon) float64 346kB -1.031e+04 -1.031e+04 ... 1.031e+04 1.031e+04
  * lat      (lat) float64 149kB -4.583e+03 -4.582e+03 ... 4.582e+03 4.583e+03
Attributes:
    long_name:     z
    actual_range:  [9.99999978e-03 3.38683990e+02]

In [12]:
ages_group = merged_dataset.to_dataframe().groupby('cluster').mean()

In [14]:
ages_group.reset_index(inplace=True)
ages_group = ages_group.groupby('age').count()

In [17]:
grouper = pd.cut(ages_group.index, 25)
ages_group.groupby(grouper).sum()

/var/folders/v7/nxggzv_j5s936v9rvl85gh2w0000gn/T/ipykernel_52378/3547488229.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ages_group.groupby(grouper).sum()


,index,cluster,z,class
"(-0.281, 13.305]",17046,17046,17046,17046
"(13.305, 26.56]",14181,14181,14181,14181
"(26.56, 39.815]",11346,11346,11346,11346
"(39.815, 53.07]",7997,7997,7997,7997
"(53.07, 66.324]",6073,6073,6073,6073
"(66.324, 79.579]",5069,5069,5069,5069
"(79.579, 92.834]",3280,3280,3280,3280
"(92.834, 106.089]",2583,2583,2583,2583
"(106.089, 119.344]",2752,2752,2752,2752
"(119.344, 132.598]",2909,2909,2909,2909


In [18]:
age_check = merged_dataset.to_dataframe().set_index('age')
age_check.groupby(grouper).size()

ValueError: Grouper and axis must be same length